# build

> Fill in a module description here

In [ ]:
#| default_exp build

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#from reproduce_work.core import read_base_config
import argparse
import toml
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
reproduce_dir = os.getenv("REPROWORKDIR", Path("./reproduce"))
dev_image_tag = os.getenv("REPRODEVIMAGE")


def read_base_config():
    with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
        base_config = toml.load(f)
    return base_config

def replace_inserts_with_toml_data(input_file, toml_file, output_file):
    # Replace \INSERT{var} with corresponding TOML data
    # Load TOML data
    with open(toml_file, 'r') as f:
        data = toml.load(f)

    # Read input text file
    with open(input_file, 'r') as f:
        content = f.read()

    start_pos = 0
    while True:
        start_idx = content.find(r'\INSERT{', start_pos)
        if start_idx == -1:
            break
        end_idx = content.find('}', start_idx)
        if end_idx == -1:
            break

        var_name = content[start_idx + 8:end_idx]
        replacement_value = data.get(var_name, '')  # Use a default value if var_name is not in TOML data
        print(f"Replacing {var_name} with {replacement_value}")
        content = content[:start_idx] + replacement_value + content[end_idx + 1:]
        start_pos = start_idx + len(replacement_value)

    # Write modified content to output .tex file
    with open(output_file, 'w') as f:
        f.write(content)

    return content





def main():

    base_config = read_base_config()

    parser = argparse.ArgumentParser(description="Replace \INSERT{var} in a text file using data from a TOML file.")
    parser.add_argument(
        "--input_file", 
        help="Path to the input text file containing \INSERT{var} placeholders.",
        default=base_config['repro']['files']['input']
    )
    parser.add_argument(
        "--toml_file", 
        help="Path to the TOML file containing replacement data.",
        default=base_config['repro']['files']['dynamic']
    )
    parser.add_argument(
        "--output_file", 
        help="Path to the output .tex file.",
        default=base_config['repro']['files']['output']
    )
    
    args = parser.parse_args()
    
    content = replace_inserts_with_toml_data(args.input_file, args.toml_file, 'reproduce/tmp/lowdown_input.md')

    print(f"Output written to {args.output_file}")


#if __name__ == "__main__":
#    main()


# Example Usage
#replace_inserts_with_toml_data('input.txt', 'data.toml', 'output.tex')


In [ ]:
#import os
#os.listdir(Path(os.getcwd()))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()